In [27]:
import mysql.connector
import pandas as pd
import numpy as np
import regex as re
from datetime import datetime

In [28]:
conn = mysql.connector.connect(
    host='212.227.90.6',
    user='Equipo19',
    password='E1q2u3i4p5o19',
    database='Equip_19'
)

cursor = conn.cursor()

cursor.execute("SHOW TABLES")
tablas = [t[0] for t in cursor.fetchall()]


dfs = {tabla: pd.read_sql(f"SELECT * FROM {tabla}", conn) for tabla in tablas}

df = dfs['Tourist_Accommodation']

C:\Users\Usuario\AppData\Local\Temp\ipykernel_1716\3720523554.py:14: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  dfs = {tabla: pd.read_sql(f"SELECT * FROM {tabla}", conn) for tabla in tablas}


In [29]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7001 entries, 0 to 7000
Data columns (total 35 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   apartment_id                 7001 non-null   int64  
 1   name                         6998 non-null   object 
 2   description                  6972 non-null   object 
 3   host_id                      7001 non-null   int64  
 4   neighbourhood_name           7001 non-null   object 
 5   neighbourhood_district       4241 non-null   object 
 6   room_type                    7001 non-null   object 
 7   accommodates                 7001 non-null   int64  
 8   bathrooms                    6969 non-null   object 
 9   bedrooms                     6972 non-null   object 
 10  beds                         6998 non-null   float64
 11  amenities_list               6984 non-null   object 
 12  price                        6870 non-null   float64
 13  minimum_nights    

CONVERSIÓN DE FECHAS

In [30]:
df["first_review_date"] = pd.to_datetime(df["first_review_date"], errors="coerce")
df["last_review_date"] = pd.to_datetime(df["last_review_date"], errors="coerce")
df["insert_date"] = pd.to_datetime(df["insert_date"], errors="coerce")

C:\Users\Usuario\AppData\Local\Temp\ipykernel_1716\3908789225.py:3: UserWarning: Parsing dates in %d/%m/%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df["insert_date"] = pd.to_datetime(df["insert_date"], errors="coerce")


CONVERSIÓN A BOOLEANO

In [31]:
df['has_availability'] = df['has_availability'].replace({'VERDADERO': True})
df['has_availability'] = df['has_availability'].astype(bool)

df['is_instant_bookable'] = df['is_instant_bookable'].replace({'VERDADERO': True, 'FALSO': False})
df['is_instant_bookable'] = df['is_instant_bookable'].astype(bool)

C:\Users\Usuario\AppData\Local\Temp\ipykernel_1716\1651559890.py:4: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['is_instant_bookable'] = df['is_instant_bookable'].replace({'VERDADERO': True, 'FALSO': False})


CONVERSIÓN A INTEGRO

In [32]:
df['bathrooms'] = pd.to_numeric(df['bathrooms'], errors='coerce').astype('Int64')
df["bedrooms"] = pd.to_numeric(df["bedrooms"], errors='coerce').astype('Int64')
df['beds'] = pd.to_numeric(df['beds'], errors='coerce').astype('Int64')

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7001 entries, 0 to 7000
Data columns (total 35 columns):
 #   Column                       Non-Null Count  Dtype         
---  ------                       --------------  -----         
 0   apartment_id                 7001 non-null   int64         
 1   name                         6998 non-null   object        
 2   description                  6972 non-null   object        
 3   host_id                      7001 non-null   int64         
 4   neighbourhood_name           7001 non-null   object        
 5   neighbourhood_district       4241 non-null   object        
 6   room_type                    7001 non-null   object        
 7   accommodates                 7001 non-null   int64         
 8   bathrooms                    6969 non-null   Int64         
 9   bedrooms                     6972 non-null   Int64         
 10  beds                         6998 non-null   Int64         
 11  amenities_list               6984 non-null 

COMPROBACIÓN DUPLICADOS APARTMENT_ID

In [33]:
df['apartment_id'].duplicated().any()

True

ELIMINACIÓN DE DUPLICADOS DE APARTMENT_ID, MANTENIENDO LOS MÁS RECIENTES SEGÚN INSERT_DATE

In [34]:
df[df['apartment_id'].duplicated(keep=False)]

df = df.sort_values(['apartment_id', 'insert_date'])

df = df.drop_duplicates(subset='apartment_id', keep='last')

In [35]:
df['apartment_id'].duplicated().any()

False

ESTANDARIZACIÓN DE CATEGORÍAS Y TEXTO

-COMO HACER ESTO????

In [36]:
df[['neighbourhood_name', 'neighbourhood_district']].head(10)

,neighbourhood_name,neighbourhood_district
0,Centro,None
1,C�rmenes,Latina
2,San Vicente,Casco Antiguo
3,l'Antiga Esquerra de l'Eixample,Eixample
4,Quart,None
5,Torroella de Fluvi�,None
6,el Camp de l'Arpa del Clot,Sant Mart�
7,la Dreta de l'Eixample,Eixample
8,Embajadores,Centro
9,el Camp d'en Grassot i Gr�cia Nova,Gr�cia


In [37]:
def clean_neighbourhood(text):
    if pd.isna(text) or text == '':
        return pd.NA
    text = str(text)
    text = text.replace("�", "")
    text = re.sub(r"[^\p{L}\p{N}\s'\-]", " ", text)
    text = re.sub(r"\s+", " ", text).strip()
    text = text.title()
    
    return text if text else pd.NA

In [38]:
df['neighbourhood_name'] = df['neighbourhood_name'].apply(clean_neighbourhood)
df['neighbourhood_district'] = df['neighbourhood_district'].apply(clean_neighbourhood)

In [39]:
df[['neighbourhood_name', 'neighbourhood_district']].head(10)

,neighbourhood_name,neighbourhood_district
0,Centro,<NA>
1,Crmenes,Latina
2,San Vicente,Casco Antiguo
3,L'Antiga Esquerra De L'Eixample,Eixample
4,Quart,<NA>
5,Torroella De Fluvi,<NA>
6,El Camp De L'Arpa Del Clot,Sant Mart
7,La Dreta De L'Eixample,Eixample
8,Embajadores,Centro
9,El Camp D'En Grassot I Grcia Nova,Grcia


ESTANDARIZAR CITY

In [40]:
df['city'].value_counts()

city
barcelona    2041
madrid       1396
mallorca     1096
girona       1085
sevilla       341
malaga        339
valencia      297
menorca       138
Name: count, dtype: int64

In [41]:
city_mapping = {
    'Seville': 'Sevilla',
    'Madrid': 'Madrid',
    'Barcelona': 'Barcelona',
    'Valencia': 'Valencia',
    'Malaga': 'Málaga',
    'Málaga': 'Málaga',        
    'València': 'Valencia',    
    'Sevilla': 'Sevilla',
    'barcelona': 'Barcelona',
    'MADRID': 'Madrid',
    'valencia': 'Valencia',
    'malaga': 'Málaga',
    'sevilla': 'Sevilla',
    'girona': 'Girona',
    'Girona': 'Girona',
    'mallorca': 'Mallorca',
    'Mallorca': 'Mallorca',
    'Menorca': 'Menorca',
    'menorca': 'Menorca',
    'madrid': 'Madrid',
    'barcelona': 'Barcelona',
    'valencia': 'Valencia',
    'malaga': 'Málaga',
    'sevilla': 'Sevilla',
    'girona': 'Girona',
    'mallorca': 'Mallorca',
    'menorca': 'Menorca',
}

In [42]:
valid_cities = {
    'Barcelona',
    'Madrid',
    'Sevilla',
    'Valencia',
    'Málaga',
    'Girona',
    'Mallorca',
    'Menorca'
}

In [43]:
df['city_clean'] = df['city'].astype(str).str.strip()

# Crea una versión en minúsculas del mapeo para la coincidencia que no distinga entre mayúsculas y minúsculas.
mapping_lower = {k.lower(): v for k, v in city_mapping.items()}

# Normalizar
df['city_normalized'] = df['city_clean'].str.lower().map(mapping_lower)

# Filtrar solo cidades válidas
df = df[df['city_normalized'].isin(valid_cities)]

# Substituir colmuna original
df['city'] = df['city_normalized']
df = df.drop(columns=['city_clean', 'city_normalized'])

In [44]:
df[['city']].head(5)

,city
0,Málaga
1,Madrid
2,Sevilla
3,Barcelona
4,Girona


ESTANDARIZAR COUNTRY

In [45]:
df[['country']].head(5)

,country
0,spain
1,spain
2,spain
3,spain
4,spain


In [46]:
df['country'] = 'Spain'
df[['country']].head(5)

,country
0,Spain
1,Spain
2,Spain
3,Spain
4,Spain


TRATAMIENTO DE VALORES NULOS

In [47]:
# Marcar inválidos
df['is_valid'] = True
df.loc[df['price'].isna() | df['city'].isna() | df['room_type'].isna(), 'is_valid'] = False
df_valid = df[df['is_valid']].copy()

# Forzar nulos en reviews si no hay reviews
review_cols = [
    'first_review_date', 'last_review_date',
    'review_scores_rating', 'review_scores_accuracy',
    'review_scores_cleanliness', 'review_scores_checkin',
    'review_scores_communication', 'review_scores_location',
    'review_scores_value', 'reviews_per_month'
]
mask_no_reviews = df_valid['number_of_reviews'] == 0
df_valid.loc[mask_no_reviews, review_cols] = np.nan

# Imputar reviews_per_month
def calculate_reviews_per_month(row):
    if row['number_of_reviews'] > 0:
        if not pd.isna(row['first_review_date']):
            today = datetime.now()
            days_since_first = (today - row['first_review_date']).days
            if days_since_first > 0:
                return row['number_of_reviews'] / (days_since_first / 30)
        elif not pd.isna(row['last_review_date']):
            today = datetime.now()
            days_since_last = (today - row['last_review_date']).days
            if days_since_last > 0:
                return row['number_of_reviews'] / (days_since_last / 30)
    return np.nan

df_valid['reviews_per_month'] = df_valid.apply(calculate_reviews_per_month, axis=1)

# Amenities_list
df_valid['amenities_list'] = df_valid['amenities_list'].apply(lambda x: [] if pd.isna(x) or (isinstance(x, str) and x.strip() == '') else x)

# Validar coherencia
review_cols_no_nulls = df_valid.loc[(df_valid['number_of_reviews'] == 0) & (df_valid[review_cols].notna().any(axis=1))]
print(f"Registros inconsistentes: {len(review_cols_no_nulls)}")

df = df_valid

Registros inconsistentes: 0


In [48]:
df

,apartment_id,name,description,host_id,neighbourhood_name,neighbourhood_district,room_type,accommodates,bathrooms,bedrooms,...,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,is_instant_bookable,reviews_per_month,country,city,insert_date,is_valid
0,11964,A ROOM WITH A VIEW,Private bedroom in our attic apartment. Right ...,45553,Centro,<NA>,Private room,2,2,1,...,100.0,100.0,100.0,100.0,False,0.406603,Spain,Málaga,2018-07-31,True
1,21853,Bright and airy room,We have a quiet and sunny room with a good vie...,83531,Crmenes,Latina,Private room,1,1,1,...,100.0,100.0,80.0,90.0,False,0.244868,Spain,Madrid,2020-01-10,True
2,32347,Explore Cultural Sights from a Family-Friendly...,Open French doors and step onto a plant-filled...,139939,San Vicente,Casco Antiguo,Entire home/apt,4,1,2,...,100.0,100.0,100.0,100.0,True,0.837578,Spain,Sevilla,2019-07-29,True
3,35379,Double 02 CasanovaRooms Barcelona,Room at a my apartment. Kitchen and 2 bathroom...,152232,L'Antiga Esquerra De L'Eixample,Eixample,Private room,2,2,1,...,100.0,100.0,100.0,90.0,True,4.287812,Spain,Barcelona,2020-01-10,True
4,35801,Can Torras Farmhouse Studio Suite,Lay in bed & watch sunlight change the mood of...,153805,Quart,<NA>,Private room,5,1,2,...,100.0,100.0,100.0,100.0,False,0.207573,Spain,Girona,2019-02-19,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6995,27237828,Studio at Finca Els Olivers,"Independent guest house, forming part of an ex...",1323233,Bscara,<NA>,Entire home/apt,2,1,1,...,100.0,100.0,100.0,100.0,False,NaN,Spain,Girona,2018-08-30,True
6996,27241318,ES MOLI D'EN SION - Villa with private pool in...,Enjoy the peace of the countryside in this bea...,80839530,Sa Pobla,<NA>,Entire home/apt,10,4,5,...,100.0,100.0,100.0,100.0,True,NaN,Spain,Mallorca,2020-04-23,True
6997,27244243,101.108_New building apartment with two double...,Apartment in Cadaqu�s center. 1rst �floor. Ele...,151496825,Cadaqus,<NA>,Entire home/apt,4,1,2,...,NaN,NaN,NaN,NaN,True,NaN,Spain,Girona,2018-08-30,True
6998,27244794,101.38_Apartment with one doble bedroom and te...,101.38.- Apartment placed Sa T�rtora � Sant An...,151496825,Cadaqus,<NA>,Entire home/apt,2,1,1,...,100.0,100.0,100.0,100.0,True,NaN,Spain,Girona,2019-12-31,True


COMPROBAR QUE MIN NIGHTS < MAX NIGHTS

In [49]:
df_error = df[df['minimum_nights'] > df['maximum_nights']]
df_error
# Sin resultados, así que todo está correcto

,apartment_id,name,description,host_id,neighbourhood_name,neighbourhood_district,room_type,accommodates,bathrooms,bedrooms,...,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,is_instant_bookable,reviews_per_month,country,city,insert_date,is_valid


COMPROBAR AVAILABILITY

In [50]:
df_hav = df[
    (df['availability_30'] > df['availability_60']) &
    (df['availability_60'] > df['availability_90']) &
    (df['availability_90'] > df['availability_365'])
].copy()

df_hav

# Sin resultados, así que todo está correcto

,apartment_id,name,description,host_id,neighbourhood_name,neighbourhood_district,room_type,accommodates,bathrooms,bedrooms,...,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,is_instant_bookable,reviews_per_month,country,city,insert_date,is_valid


In [51]:
cols_disp = ['availability_30', 'availability_60', 'availability_90', 'availability_365']

condicion_invalida = (
    (df['has_availability'] == True) & 
    (df[cols_disp].isna().all(axis=1))
)

df_invalidas = df[condicion_invalida]
print(df_invalidas)

# Empty Dataframe, así que todo está correcto

Empty DataFrame
Columns: [apartment_id, name, description, host_id, neighbourhood_name, neighbourhood_district, room_type, accommodates, bathrooms, bedrooms, beds, amenities_list, price, minimum_nights, maximum_nights, has_availability, availability_30, availability_60, availability_90, availability_365, number_of_reviews, first_review_date, last_review_date, review_scores_rating, review_scores_accuracy, review_scores_cleanliness, review_scores_checkin, review_scores_communication, review_scores_location, review_scores_value, is_instant_bookable, reviews_per_month, country, city, insert_date, is_valid]
Index: []

[0 rows x 36 columns]


LIMPIEZA AVAILABILITY

In [52]:
df_disponibilidad = df.loc[df['has_availability'] != True, 
                     ['has_availability', 'availability_30', 'availability_60', 'availability_90', 'availability_365']]

df_disponibilidad

,has_availability,availability_30,availability_60,availability_90,availability_365
7,False,28,55,85,360
12,False,7,23,53,328
29,False,13,43,72,347
32,False,5,11,37,234
36,False,8,14,25,224
...,...,...,...,...,...
4762,False,16,46,76,76
4771,False,6,15,31,114
4791,False,13,43,73,163
4802,False,3,33,63,338


In [53]:
sidisponible = (df[cols_disp] > 0).any(axis=1)

df.loc[(df['has_availability'] == False) & sidisponible, 'has_availability'] = True

In [54]:
df[['has_availability'] + cols_disp].head()

,has_availability,availability_30,availability_60,availability_90,availability_365
0,True,7,20,40,130
1,True,0,0,0,162
2,True,26,31,31,270
3,True,9,23,49,300
4,True,0,19,49,312


COHERENCIA FECHAS INSERT_DATE ANTES QUE FIRST_REVIEW_DATE

In [55]:
df_fechas = df[
    (df['insert_date'] > df['first_review_date']) 
].copy()

totaldf = len(df_fechas)
totaldf

df_fechas

,apartment_id,name,description,host_id,neighbourhood_name,neighbourhood_district,room_type,accommodates,bathrooms,bedrooms,...,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,is_instant_bookable,reviews_per_month,country,city,insert_date,is_valid
0,11964,A ROOM WITH A VIEW,Private bedroom in our attic apartment. Right ...,45553,Centro,<NA>,Private room,2,2,1,...,100.0,100.0,100.0,100.0,False,0.406603,Spain,Málaga,2018-07-31,True
1,21853,Bright and airy room,We have a quiet and sunny room with a good vie...,83531,Crmenes,Latina,Private room,1,1,1,...,100.0,100.0,80.0,90.0,False,0.244868,Spain,Madrid,2020-01-10,True
2,32347,Explore Cultural Sights from a Family-Friendly...,Open French doors and step onto a plant-filled...,139939,San Vicente,Casco Antiguo,Entire home/apt,4,1,2,...,100.0,100.0,100.0,100.0,True,0.837578,Spain,Sevilla,2019-07-29,True
4,35801,Can Torras Farmhouse Studio Suite,Lay in bed & watch sunlight change the mood of...,153805,Quart,<NA>,Private room,5,1,2,...,100.0,100.0,100.0,100.0,False,0.207573,Spain,Girona,2019-02-19,True
5,48764,18th C Stone House near Costa Brava,Casa Fluvia is a charming stone village house ...,220145,Torroella De Fluvi,<NA>,Entire home/apt,8,2,4,...,100.0,100.0,90.0,100.0,False,0.140819,Spain,Girona,2019-02-19,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6980,27191914,Habitaci�n privada con localizaci�n buenissima,"Habitaci�n privada super bien situada, con ba�...",203782616,El Poble Sec,Sants-Montjuc,Private room,1,1,1,...,90.0,90.0,90.0,90.0,False,0.021444,Spain,Barcelona,2021-01-12,True
6981,27197045,Soho puerto 2,Disfruta de un apartamento moderno y c�modo re...,120568489,Centro,<NA>,Entire home/apt,6,1,2,...,90.0,90.0,100.0,80.0,True,0.214439,Spain,Málaga,2020-03-26,True
6984,27212538,NEW Modernist balcony in Ps de Gr�cia,Apartment with 2 double rooms and maximum capa...,127482361,La Dreta De L'Eixample,Eixample,Entire home/apt,4,1,2,...,100.0,100.0,100.0,90.0,True,0.039894,Spain,Barcelona,2020-03-16,True
6987,27222952,"Villa Can Mart�, Piscina, relax y naturaleza.",Ca�n Mart� es una hermosa y confortable villa ...,173406079,Sa Pobla,<NA>,Entire home/apt,6,2,3,...,100.0,100.0,100.0,100.0,True,0.011095,Spain,Mallorca,2019-03-25,True


COHERENCIA FECHAS FIRST_REVIEW DATE ANTES QUE LAST_REVIEW_DATE

In [56]:
df_fechas2 = df[
    (df['first_review_date'] > df['last_review_date'])
].copy()

totaldf2 = len(df_fechas2)
totaldf2

df_fechas2[['first_review_date', 'last_review_date']]

,first_review_date,last_review_date
596,2017-12-04,2017-01-05
797,2016-08-02,2016-06-06
969,2015-03-01,2015-02-07
1148,2017-07-05,2017-01-11
1166,2015-05-07,2015-01-08
1177,2016-07-01,2016-06-09
1461,2016-10-07,2016-03-08
1782,2016-09-04,2016-05-06
1880,2016-05-01,2016-02-09
1906,2016-12-02,2016-02-08


In [57]:
def arreglar_fechas_reviews(row):
    if row['first_review_date'] > row['last_review_date']:
        row['first_review_date'], row['last_review_date'] = row['last_review_date'], row['first_review_date']
    return row

df = df.apply(arreglar_fechas_reviews, axis=1)

CORRECCIÓN INSTANT_BOOKABLE

In [58]:
isavailable = (df['has_availability'] == True) & (df['price'] > 0)

df_incoherentes = df[(df['is_instant_bookable'] == True) & (~isavailable)]

df_incoherentes[['apartment_id', 'has_availability', 'price', 'is_instant_bookable']]
print(f"{len(df_incoherentes)} filas no cumplen las condiciones.")

4 filas no cumplen las condiciones.


In [59]:
df[['price', 'has_availability', 'is_instant_bookable']]

,price,has_availability,is_instant_bookable
0,400.0,True,False
1,170.0,True,False
2,990.0,True,True
3,400.0,True,True
4,900.0,True,False
...,...,...,...
6995,1500.0,True,False
6996,3130.0,True,True
6997,990.0,True,True
6998,720.0,True,True


In [60]:
df['is_instant_bookable'] = np.where(
    (df['price'] > 0) & (df['has_availability'] == True),
    True,
    False
)

CAMBIO DE SCORES RATING A 0 - 100 Y TODOS LOS DEMÁS A 0 - 10

In [61]:
df['review_scores_rating'] = df['review_scores_rating'] / 10
df['review_scores_accuracy'] = df['review_scores_accuracy'] / 10
df['review_scores_checkin'] = df['review_scores_checkin'] / 10
df['review_scores_cleanliness'] = df['review_scores_cleanliness'] / 10
df['review_scores_communication'] = df['review_scores_communication'] / 10
df['review_scores_location'] = df['review_scores_location'] / 10
df['review_scores_value'] = df['review_scores_value'] / 10


In [62]:
df['review_scores_value'].head(10)

0    10.0
1     9.0
2    10.0
3     9.0
4    10.0
5    10.0
6     9.0
7     9.0
8     9.0
9     9.0
Name: review_scores_value, dtype: float64

ELIMINACIÓN NULOS EN BEDROOMS

In [63]:
df.loc[df['bedrooms'].isnull(), ['bedrooms', 'room_type', 'accommodates']]

,bedrooms,room_type,accommodates
614,<NA>,Entire home/apt,3
859,<NA>,Entire home/apt,2
995,<NA>,Entire home/apt,2
1126,<NA>,Entire home/apt,2
1196,<NA>,Entire home/apt,2
1228,<NA>,Entire home/apt,2
1356,<NA>,Entire home/apt,6
1381,<NA>,Entire home/apt,4
1871,<NA>,Entire home/apt,3
1891,<NA>,Entire home/apt,4


In [64]:
df.loc[(df['room_type'] == 'Private room') & (df['bedrooms'].isnull()), 'bedrooms'] = 1
df.loc[(df['room_type'] == 'Entire home/apt') & (df['bedrooms'].isnull()), 'bedrooms'] = 0

In [65]:
df[['bedrooms', 'room_type', 'accommodates']]

,bedrooms,room_type,accommodates
0,1,Private room,2
1,1,Private room,1
2,2,Entire home/apt,4
3,1,Private room,2
4,2,Private room,5
...,...,...,...
6995,1,Entire home/apt,2
6996,5,Entire home/apt,10
6997,2,Entire home/apt,4
6998,1,Entire home/apt,2


ELIMINACIÓN NULOS EN BATHROOMS

In [66]:
df.loc[df['bathrooms'].isnull(), ['bathrooms', 'room_type', 'accommodates']]
df.loc[df['bathrooms'].isnull(), 'bathrooms'] = 1

ELIMINACIÓN NULOS EN BEDS

In [67]:
df.loc[df['beds'].isnull(), ['beds', 'bedrooms','room_type', 'accommodates']]
df.loc[df['beds'].isnull(), 'beds'] = df['bedrooms']

ELIMINACIÓN DE NULOS EN REVIEWS

In [68]:
df.loc[df['reviews_per_month'].isnull(), ['reviews_per_month', 'number_of_reviews']]

,reviews_per_month,number_of_reviews
8,NaN,108
26,NaN,97
27,NaN,44
28,NaN,189
30,NaN,33
...,...,...
6992,NaN,0
6995,NaN,1
6996,NaN,1
6997,NaN,0


In [69]:
df.loc[
    (df['reviews_per_month'].isnull()) & (df['number_of_reviews'] == 0),
    'reviews_per_month'
] = 0

In [70]:
df[
    ['reviews_per_month', 'number_of_reviews', 'first_review_date', 'last_review_date', 'review_scores_rating',
    'review_scores_accuracy', 'review_scores_checkin', 'review_scores_cleanliness', 'review_scores_communication',
    'review_scores_location', 'review_scores_value']]

,reviews_per_month,number_of_reviews,first_review_date,last_review_date,review_scores_rating,review_scores_accuracy,review_scores_checkin,review_scores_cleanliness,review_scores_communication,review_scores_location,review_scores_value
0,0.406603,78,2010-02-01,2017-05-09,97.0,10.0,10.0,10.0,10.0,10.0,10.0
1,0.244868,33,2014-10-10,NaT,92.0,9.0,10.0,9.0,10.0,8.0,9.0
2,0.837578,148,2011-05-01,NaT,98.0,10.0,10.0,10.0,10.0,10.0,10.0
3,4.287812,292,NaT,2020-04-01,94.0,10.0,10.0,9.0,10.0,10.0,9.0
4,0.207573,36,2011-08-07,2018-08-08,97.0,10.0,10.0,10.0,10.0,10.0,10.0
...,...,...,...,...,...,...,...,...,...,...,...
6995,NaN,1,NaT,NaT,100.0,10.0,10.0,10.0,10.0,10.0,10.0
6996,NaN,1,NaT,NaT,100.0,10.0,10.0,10.0,10.0,10.0,10.0
6997,0.000000,0,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6998,NaN,1,NaT,NaT,100.0,10.0,10.0,10.0,10.0,10.0,10.0


In [71]:
df.loc[df['number_of_reviews'] == 0, 
       ['reviews_per_month', 
        'number_of_reviews', 
        'first_review_date', 
        'last_review_date', 
        'review_scores_rating', 
        'review_scores_accuracy', 
        'review_scores_checkin', 
        'review_scores_cleanliness', 
        'review_scores_communication', 
        'review_scores_location', 
        'review_scores_value']]

,reviews_per_month,number_of_reviews,first_review_date,last_review_date,review_scores_rating,review_scores_accuracy,review_scores_checkin,review_scores_cleanliness,review_scores_communication,review_scores_location,review_scores_value
61,0.0,0,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
103,0.0,0,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
137,0.0,0,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
142,0.0,0,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
168,0.0,0,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
6988,0.0,0,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6990,0.0,0,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6991,0.0,0,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6992,0.0,0,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [72]:
colsrev = [
    'reviews_per_month',
    'number_of_reviews',
    'review_scores_rating',
    'review_scores_accuracy',
    'review_scores_checkin',
    'review_scores_cleanliness',
    'review_scores_communication',
    'review_scores_location',
    'review_scores_value'
]

df.loc[df['number_of_reviews'] == 0, colsrev] = 0

In [73]:
df.isnull().sum()

apartment_id                      0
name                              3
description                      26
host_id                           0
neighbourhood_name                0
neighbourhood_district         2577
room_type                         0
accommodates                      0
bathrooms                         0
bedrooms                          0
beds                              0
amenities_list                    0
price                             0
minimum_nights                    0
maximum_nights                    0
has_availability                  0
availability_30                   0
availability_60                   0
availability_90                   0
availability_365                  0
number_of_reviews                 0
first_review_date              4339
last_review_date               4378
review_scores_rating             74
review_scores_accuracy           83
review_scores_cleanliness        77
review_scores_checkin            88
review_scores_communication 

¿FIRST_REVIEW_DATE Y LAST_REVIEW_DATE?

In [74]:
# df.loc[
#     df['last_review_date'].isnull() & df['first_review_date'].notnull(),
#     ['first_review_date', 'last_review_date', 'number_of_reviews', 'reviews_per_month']
# ]

df.loc[
    df['first_review_date'].isnull() & df['last_review_date'].notnull(),
    ['first_review_date', 'last_review_date', 'number_of_reviews', 'reviews_per_month']
]

,first_review_date,last_review_date,number_of_reviews,reviews_per_month
3,NaT,2020-04-01,292,4.287812
7,NaT,2017-05-06,33,0.318943
11,NaT,2016-01-10,189,1.581149
19,NaT,2017-09-12,154,1.552941
32,NaT,2017-02-04,416,3.906103
...,...,...,...,...
6963,NaT,2018-07-09,3,0.033645
6972,NaT,2020-07-03,37,0.569231
6982,NaT,2020-05-10,95,1.422156
6985,NaT,2019-11-03,22,0.300958


CREACIÓN DE NUEVA COLUMNA

In [75]:
df['ocupacion_mes'] = 30 - df['availability_30']

In [76]:
df

,apartment_id,name,description,host_id,neighbourhood_name,neighbourhood_district,room_type,accommodates,bathrooms,bedrooms,...,review_scores_communication,review_scores_location,review_scores_value,is_instant_bookable,reviews_per_month,country,city,insert_date,is_valid,ocupacion_mes
0,11964,A ROOM WITH A VIEW,Private bedroom in our attic apartment. Right ...,45553,Centro,<NA>,Private room,2,2,1,...,10.0,10.0,10.0,True,0.406603,Spain,Málaga,2018-07-31,True,23
1,21853,Bright and airy room,We have a quiet and sunny room with a good vie...,83531,Crmenes,Latina,Private room,1,1,1,...,10.0,8.0,9.0,True,0.244868,Spain,Madrid,2020-01-10,True,30
2,32347,Explore Cultural Sights from a Family-Friendly...,Open French doors and step onto a plant-filled...,139939,San Vicente,Casco Antiguo,Entire home/apt,4,1,2,...,10.0,10.0,10.0,True,0.837578,Spain,Sevilla,2019-07-29,True,4
3,35379,Double 02 CasanovaRooms Barcelona,Room at a my apartment. Kitchen and 2 bathroom...,152232,L'Antiga Esquerra De L'Eixample,Eixample,Private room,2,2,1,...,10.0,10.0,9.0,True,4.287812,Spain,Barcelona,2020-01-10,True,21
4,35801,Can Torras Farmhouse Studio Suite,Lay in bed & watch sunlight change the mood of...,153805,Quart,<NA>,Private room,5,1,2,...,10.0,10.0,10.0,True,0.207573,Spain,Girona,2019-02-19,True,30
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6995,27237828,Studio at Finca Els Olivers,"Independent guest house, forming part of an ex...",1323233,Bscara,<NA>,Entire home/apt,2,1,1,...,10.0,10.0,10.0,True,NaN,Spain,Girona,2018-08-30,True,8
6996,27241318,ES MOLI D'EN SION - Villa with private pool in...,Enjoy the peace of the countryside in this bea...,80839530,Sa Pobla,<NA>,Entire home/apt,10,4,5,...,10.0,10.0,10.0,True,NaN,Spain,Mallorca,2020-04-23,True,4
6997,27244243,101.108_New building apartment with two double...,Apartment in Cadaqu�s center. 1rst �floor. Ele...,151496825,Cadaqus,<NA>,Entire home/apt,4,1,2,...,0.0,0.0,0.0,True,0.000000,Spain,Girona,2018-08-30,True,6
6998,27244794,101.38_Apartment with one doble bedroom and te...,101.38.- Apartment placed Sa T�rtora � Sant An...,151496825,Cadaqus,<NA>,Entire home/apt,2,1,1,...,10.0,10.0,10.0,True,NaN,Spain,Girona,2019-12-31,True,30


COMPROBACIÓN NULOS DE NEIGHBOURHOOD DISTRICT

In [77]:
df.loc[df["neighbourhood_district"].isna(), "neighbourhood_name"].value_counts()

df_nulls = df[df["neighbourhood_district"].isna()][["neighbourhood_name", "city", "apartment_id"]]

df_nulls

,neighbourhood_name,city,apartment_id
0,Centro,Málaga,11964
4,Quart,Girona,35801
5,Torroella De Fluvi,Girona,48764
27,Palma De Mallorca,Mallorca,164621
30,Alcdia,Mallorca,178456
...,...,...,...
6995,Bscara,Girona,27237828
6996,Sa Pobla,Mallorca,27241318
6997,Cadaqus,Girona,27244243
6998,Cadaqus,Girona,27244794


In [78]:
df[["neighbourhood_district", "neighbourhood_name"]]

,neighbourhood_district,neighbourhood_name
0,<NA>,Centro
1,Latina,Crmenes
2,Casco Antiguo,San Vicente
3,Eixample,L'Antiga Esquerra De L'Eixample
4,<NA>,Quart
...,...,...
6995,<NA>,Bscara
6996,<NA>,Sa Pobla
6997,<NA>,Cadaqus
6998,<NA>,Cadaqus


COMPROBACIÓN OUTLIERS

In [79]:
df.head()

,apartment_id,name,description,host_id,neighbourhood_name,neighbourhood_district,room_type,accommodates,bathrooms,bedrooms,...,review_scores_communication,review_scores_location,review_scores_value,is_instant_bookable,reviews_per_month,country,city,insert_date,is_valid,ocupacion_mes
0,11964,A ROOM WITH A VIEW,Private bedroom in our attic apartment. Right ...,45553,Centro,<NA>,Private room,2,2,1,...,10.0,10.0,10.0,True,0.406603,Spain,Málaga,2018-07-31,True,23
1,21853,Bright and airy room,We have a quiet and sunny room with a good vie...,83531,Crmenes,Latina,Private room,1,1,1,...,10.0,8.0,9.0,True,0.244868,Spain,Madrid,2020-01-10,True,30
2,32347,Explore Cultural Sights from a Family-Friendly...,Open French doors and step onto a plant-filled...,139939,San Vicente,Casco Antiguo,Entire home/apt,4,1,2,...,10.0,10.0,10.0,True,0.837578,Spain,Sevilla,2019-07-29,True,4
3,35379,Double 02 CasanovaRooms Barcelona,Room at a my apartment. Kitchen and 2 bathroom...,152232,L'Antiga Esquerra De L'Eixample,Eixample,Private room,2,2,1,...,10.0,10.0,9.0,True,4.287812,Spain,Barcelona,2020-01-10,True,21
4,35801,Can Torras Farmhouse Studio Suite,Lay in bed & watch sunlight change the mood of...,153805,Quart,<NA>,Private room,5,1,2,...,10.0,10.0,10.0,True,0.207573,Spain,Girona,2019-02-19,True,30


In [80]:
pd.set_option('display.max_columns', None)
df[df['price'] > 6000]

,apartment_id,name,description,host_id,neighbourhood_name,neighbourhood_district,room_type,accommodates,bathrooms,bedrooms,beds,amenities_list,price,minimum_nights,maximum_nights,has_availability,availability_30,availability_60,availability_90,availability_365,number_of_reviews,first_review_date,last_review_date,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,is_instant_bookable,reviews_per_month,country,city,insert_date,is_valid,ocupacion_mes
2390,12970898,APARTAMENTO CENTRICO-EL CARMEN,Apartamento totalmente reformado en 2018. Dos ...,71363084,La Xerea,Ciutat Vella,Entire home/apt,6,2,3,7,"Paid parking on premises, Shampoo, Oven, Host ...",6071.0,2,60,True,0,0,3,3,38,2018-09-03,NaT,93.0,10.0,9.0,10.0,10.0,10.0,9.0,True,0.435281,Spain,Valencia,2020-09-28,True,30


In [81]:
pd.set_option('display.max_columns', None)
df[df['beds'] > 25]

,apartment_id,name,description,host_id,neighbourhood_name,neighbourhood_district,room_type,accommodates,bathrooms,bedrooms,beds,amenities_list,price,minimum_nights,maximum_nights,has_availability,availability_30,availability_60,availability_90,availability_365,number_of_reviews,first_review_date,last_review_date,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,is_instant_bookable,reviews_per_month,country,city,insert_date,is_valid,ocupacion_mes


In [82]:
# df.to_csv('tourist_accommodation_30102025_clean.csv', index=False)  